In [30]:
from utils import read_lines
from collections import defaultdict

deltas = {
    'up': (-1, 0),
    'down': (1, 0),
    'left': (0, -1),
    'right': (0, 1),
}

left_turn = {
    'up': 'left',
    'left': 'down',
    'down': 'right',
    'right': 'up',
}

right_turn = {
    'up': 'right',
    'right': 'down',
    'down': 'left',
    'left': 'up',
}

def move(pos, dirction):
    y, x = pos
    dy, dx = deltas[dirction]
    return (y+dy, x+dx)

def parse_program(s):
    return [int(x) for x in s.split(',')]

def parse_code(num):
    if num == 99:
        return num, [0]
    else:
        digits = []
        while num > 0:
            digits.append(num % 10)
            num //=10
        digits += [0] * (5-len(digits))
        opcode = digits[0]
        param_modes = digits[2:]
        return opcode, param_modes

def run(program, input=0):
    i = 0
    cur_output = []
    outputs = []
    extend_memory = defaultdict(int)
    relative_base = 0

    cur_pos = (0, 0)
    cur_direction = 'up'
    painted = defaultdict(int)
    painted[cur_pos] = input

    def read_memory(addr):
        if addr < len(program):
            return program[addr]
        else:
            return extend_memory[addr]
        
    def write_memory(idx, value, mode):
        addr = program[idx]
        if mode == 2:
            addr = relative_base + addr
        if addr < len(program):
            program[addr] = value
        else:
            extend_memory[addr] = value
    
    def get_oprand(idx, mode):
        if mode == 1:
            return program[idx]
        if mode == 2:
            addr = relative_base + program[idx]
        else:
            addr = program[idx]
        return read_memory(addr)

    while i < len(program):
        opcode, param_modes = parse_code(program[i])
        if opcode in (1, 2, 5, 6, 7, 8):
            oprand1 = get_oprand(i+1, param_modes[0])
            oprand2 = get_oprand(i+2, param_modes[1])
    
            if opcode == 1:
                write_memory(i+3, oprand1 + oprand2, param_modes[2])
                i+=4
            elif opcode == 2:
                write_memory(i+3, oprand1 * oprand2, param_modes[2])
                i+=4
            elif opcode == 5:
                if oprand1 != 0:
                    i = oprand2
                else:
                    i += 3
            elif opcode == 6:
                if oprand1 == 0:
                    i = oprand2
                else:
                    i += 3
            elif opcode == 7:
                write_memory(i+3, 1 if oprand1 < oprand2 else 0, param_modes[2])
                i+=4
            elif opcode == 8:
                write_memory(i+3, 1 if oprand1 == oprand2 else 0, param_modes[2])
                i+=4
        elif opcode ==3 :
            write_memory(i+1, painted[cur_pos], param_modes[0])
            i += 2
        elif opcode == 4:
            oprand1 = get_oprand(i+1, param_modes[0])
            cur_output.append(oprand1)
            assert oprand1 in (0, 1)
            if len(cur_output) == 1:
                painted[cur_pos] = oprand1
            else:
                outputs.append(cur_output)
                cur_output = []
                if oprand1 == 0:
                    cur_direction = left_turn[cur_direction]
                else:
                    cur_direction = right_turn[cur_direction]
                cur_pos = move(cur_pos, cur_direction)
            i += 2
        elif opcode == 9:
            oprand1 = get_oprand(i+1, param_modes[0])
            relative_base += oprand1
            i+=2
        elif program[i] == 99:
            return outputs, painted
        else:
            raise ValueError(f'illegal code {i} {program[i]}')
        
def part1(input_file):
    s = read_lines(input_file)[0]
    program = parse_program(s)
    outputs, painted = run(program)
    return len(painted)

def part2(input_file):
    s = read_lines(input_file)[0]
    program = parse_program(s)
    outputs, painted = run(program, 1)
    # print(outputs)
    # print(painted)
    min_row, max_row, min_col, max_col = 0, 0, 0, 0
    for r, c in painted:
        min_row = min(r, min_row)
        max_row = max(r, max_row)
        min_col = min(c, min_col)
        max_col = max(c, max_col)
    height, width = max_row - min_row + 1, max_col - min_col + 1
    board = [[0] * width for _ in range(height)]
    for (y, x), c in painted.items():
        board[y-min_row][x-min_col] = c
    for row in board:
        print(''.join([str(c) for c in row]))

In [31]:
part1('inputs/day11.txt')

1894

In [32]:
part2('inputs/day11.txt')

0001101001011110100001111000110111001001000
0000101010000010100000001000010100101001000
0000101100000100100000010000010111001111000
0000101010001000100000100000010100101001000
0100101010010000100001000010010100101001000
0011001001011110111101111001100111001001000


In [33]:
print('JKZLZJBH')

JKZLZJBH
